In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

2023-07-24 15:49:50.248211: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# import the data set
df = pd.read_csv("Churn_Modelling.csv")

In [5]:
#Show the dataset
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [6]:
#convert categorical features to label  encoded features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
# print after changes of lavel encoding 
print(X[0:10,:])

[[619 'France' 0 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 0 42 8 159660.8 3 1 0 113931.57]
 [699 'France' 0 39 1 0.0 2 0 0 93826.63]
 [850 'Spain' 0 43 2 125510.82 1 1 1 79084.1]
 [645 'Spain' 1 44 8 113755.78 2 1 0 149756.71]
 [822 'France' 1 50 7 0.0 2 1 1 10062.8]
 [376 'Germany' 0 29 4 115046.74 4 1 0 119346.88]
 [501 'France' 1 44 4 142051.07 2 0 1 74940.5]
 [684 'France' 1 27 2 134603.88 1 1 1 71725.73]]


In [7]:
# one hot encoding now 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# encode is name of transformer
# onehotencoder is type of transformer 
# [1] is column to transofmr
# passthrough says whatver else , keep as it is
ct = ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X[0:10,:])


[[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]
 [0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]
 [1.0 0.0 0.0 502 0 42 8 159660.8 3 1 0 113931.57]
 [1.0 0.0 0.0 699 0 39 1 0.0 2 0 0 93826.63]
 [0.0 0.0 1.0 850 0 43 2 125510.82 1 1 1 79084.1]
 [0.0 0.0 1.0 645 1 44 8 113755.78 2 1 0 149756.71]
 [1.0 0.0 0.0 822 1 50 7 0.0 2 1 1 10062.8]
 [0.0 1.0 0.0 376 0 29 4 115046.74 4 1 0 119346.88]
 [1.0 0.0 0.0 501 1 44 4 142051.07 2 0 1 74940.5]
 [1.0 0.0 0.0 684 1 27 2 134603.88 1 1 1 71725.73]]


In [8]:
# Split to test , train , validation sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) 

In [9]:
print("Og dataset ",X.shape)
print("Training dataset ",X_train.shape)
print("Test dataset ",X_test.shape)

Og dataset  (10000, 12)
Training dataset  (8000, 12)
Test dataset  (2000, 12)


In [10]:
# Scaling features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
print(X_train[0:1,:])

[[-1.01460667 -0.5698444   1.74309049  0.16958176 -1.09168714 -0.46460796
   0.00666099 -1.21571749  0.8095029   0.64259497 -1.03227043  1.10643166]]
(8000, 12)


In [11]:
#Building ANN 
ann = tf.keras.models.Sequential([
    tf.keras.Input(shape=(12,)),
    tf.keras.layers.Dense(units=12 , activation='relu'),
    tf.keras.layers.Dense(units=20 , activation='relu'),
    tf.keras.layers.Dense(units=1 , activation='sigmoid'),
])

In [12]:
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                156       
                                                                 
 dense_1 (Dense)             (None, 20)                260       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 437 (1.71 KB)
Trainable params: 437 (1.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
#compile the ann

ann.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
#train the model by fitting to dataset
ann.fit(X_train,y_train,epochs=100,batch_size=32)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.4224 - accuracy: 0.8231
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3576 - accuracy: 0.8521
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3495 - accuracy: 0.8574
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3466 - accuracy: 0.8586
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3445 - accuracy: 0.8614
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3410 - accuracy: 0.8611
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3391 - accuracy: 0.8624
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3392 - accuracy: 0.8609
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3381 - accuracy: 0.8626
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3362 - accura

In [15]:
def predictCategory(probability):
    if probability <0.5:
        print("Will Not Exit")
    else:
        print("Will Exit")

In [16]:
print("train dataset prediction :")
print(X_train[1:2,:].shape)
# print(X_test.shape)
predictCategory(ann.predict(X_train[1:2,:])[0][0])
print("train dataset actual :")
predictCategory(y_train[0])

train dataset prediction :
(1, 12)
1/1 [==============================] - 0s 78ms/step
Will Not Exit
train dataset actual :
Will Not Exit


In [17]:

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

#why reshape???
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [1 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [18]:
#confusion matrix 

from sklearn.metrics import confusion_matrix, accuracy_score 
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1533   62]
 [ 226  179]]


0.856

In [19]:
truePositive = cm[0][0]
falsePositive = cm[0][1]
falseNegative = cm[0][1]
print("precision =",truePositive/(truePositive+falsePositive))
print("recall =",truePositive/(truePositive+falseNegative))

precision = 0.961128526645768
recall = 0.961128526645768
